In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

# Set device to CPU (or GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dataset directory (all data inside one folder with class subfolders)
data_dir = 'C:/Users/GENAIKOLGPUSR32/Downloads/archive (1)/validation/train'  # Adjust path accordingly

# Data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])  # ImageNet normalization
])

# Dataset and loader
dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0)

# Number of classes
num_classes = len(dataset.classes)
print(f'Number of classes: {num_classes}')

# Build model with pretrained ResNet50
#model = models.resnet50(pretrained=True)
model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace final classifier
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, num_classes)
)

model = model.to(device)

# Loss and optimizer (only training final layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop (no validation)
num_epochs = 1

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    model.train()
    running_loss = 0.0
    running_corrects = 0
    
    for inputs, labels in tqdm(dataloader, desc='Training batches'):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / len(dataset)
    epoch_acc = running_corrects.double() / len(dataset)
    print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    

Number of classes: 6
Epoch 1/1


Training batches: 100%|███████████████████████████████████████████████████████████████████████████| 142/142 [07:36<00:00,  3.21s/it]

Train Loss: 1.1456 Acc: 0.5871


In [7]:
!pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
torch.save(model, 'plant_disease_full_model.pth')

#model = torch.load('plant_disease_full_model.pth')
#model.eval()

In [15]:
model.load_state_dict(torch.load('plant_disease_full_model.pth', map_location=torch.device('cpu')))

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL torchvision.models.resnet.ResNet was not an allowed global by default. Please use `torch.serialization.add_safe_globals([torchvision.models.resnet.ResNet])` or the `torch.serialization.safe_globals([torchvision.models.resnet.ResNet])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load 

In [16]:
import torch
import torchvision.models.resnet

torch.serialization.add_safe_globals([torchvision.models.resnet.ResNet])

model = torch.load('plant_disease_full_model.pth', weights_only=False)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [21]:
from PIL import Image
from torchvision import transforms

def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),        # Resize to model input size
        transforms.ToTensor(),                 # Convert to tensor
        transforms.Normalize(                  # Normalize with ImageNet stats
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image



image_path = 'C:/Users/GENAIKOLGPUSR32/Downloads/archive (1)/Dataset/Tomato_healthy/6d7934cb-69fc-48fe-b493-a64b349a9094___RS_HL 9930.jpg'
input_tensor = preprocess_image(image_path)

with torch.no_grad():
    outputs = model(input_tensor)
    _, predicted = torch.max(outputs, 1)

# Map predicted class index to label
class_names = ['Apple___Cedar_apple_rust', 'Apple___healthy', 'Potato___Late_blight', 'Potato___healthy','Tomato__Tomato_mosaic_virus','Tomato_healthy']  # replace with your classes
predicted_class = class_names[predicted.item()]

print(f"Predicted class: {predicted_class}")

Predicted class: Tomato_healthy


In [18]:
dataset.classes

['Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato__Tomato_mosaic_virus',
 'Tomato_healthy']